In [1]:
# Imports

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np
import csv
import os
import pyedflib
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from imblearn.over_sampling import BorderlineSMOTE
import pickle
from torch.utils.data import Dataset


In [2]:

folder = 'ISRUC DATASET SUBGROUP 1 EXTRACTED'
signals_to_check = 8
no_of_subjects = 30
sensor_values_count = 5280000
labels_values_count = 880
frequency_of_each_sensor = 200 #In hertz means (200 values in 1 second)
epoch_size = 30 #In Seconds

#Awake
Awake = 0
#Non Rapid Eye Movement
N1 = 1
N2 = 2
N3 = 3
#Rapid Eye Movement
R = 5


# Data Loading

In [8]:
# After Executing Ram Usage 12.6GB
# Save Memory Loading
start_time = time.time()

X_tensor = None
y_tensor = None

for i in tqdm(range(1, no_of_subjects + 1)):
    subject_path = os.path.join(folder, str(i))

    # Load reshaped data and labels directly into tensors
    input_data = torch.tensor(np.load(os.path.join(subject_path, "input.npy")), dtype=torch.float32)
    labels = torch.tensor(np.load(os.path.join(subject_path, "labels.npy")), dtype=torch.int8)

    # Concatenate data to tensors
    if X_tensor is None:
        X_tensor = input_data
        y_tensor = labels
    else:
        X_tensor = torch.cat((X_tensor, input_data), dim=0)
        y_tensor = torch.cat((y_tensor, labels), dim=0)

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X_tensor))
train_input_data, test_input_data = X_tensor[:split_index], X_tensor[split_index:]
train_labels, test_labels = y_tensor[:split_index], y_tensor[split_index:]

end_time = time.time()
elapsed_time = end_time - start_time
print("Time to execute: ", elapsed_time)


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [01:51<00:00,  3.71s/it]

Time to execute:  111.26752758026123


In [9]:
# We have 
# Train data = 14080    1 Batch = 14080/batch_size
# Test data = 3520      1 Batch = 3520/batch_size

class MyCustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 32
num_epochs = 100
# Convert data to PyTorch tensors
#X_tensor = torch.tensor(all_input_data, dtype=torch.float32)
#y_tensor = torch.tensor(all_labels, dtype=torch.int32) #Previously using int32

# Define dataloaders for training and testing
dataset = TensorDataset(X_tensor, y_tensor)
#train_size = int(0.8 * len(dataset))
#test_size = len(dataset) - train_size
#train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#train_features, train_labels = next(iter(train_loader))


 # First split to get training (80%) and remaining (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, stratify=y_tensor)

# Further split remaining into validation (10%) and test (10%)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, stratify=y_train_val)

# Create datasets
train_dataset = MyCustomDataset(X_train, y_train)
val_dataset = MyCustomDataset(X_val, y_val)
test_dataset = MyCustomDataset(X_test, y_test)

test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
torch.cuda.empty_cache()
device

device(type='cuda')

In [ ]:
# First run training  on unbalanced dataset
# Apply data augmentation to balanced the dataset 
# Training on balanced dataset 
# epoch size 100
# Folds 5
# Without Bsmote => Noise Injection
# Data Augmentation 

# MODEL STRUCTURE

In [5]:
class SleepStageClassifier(nn.Module):
    def __init__(self, input_channels=8, num_classes=6):
        super(SleepStageClassifier, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Batch normalization layers
        self.batchnorm1 = nn.BatchNorm1d(16)
        self.batchnorm2 = nn.BatchNorm1d(32)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
        # Max pooling layers
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * (6000 // 8), 1024)  # Adjust input size based on the actual input shape
        self.fc2 = nn.Linear(1024, num_classes)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Convolutional layers with batch normalization, ReLU activation, and max pooling
        
        x = self.pool(nn.functional.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool(nn.functional.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool(nn.functional.relu(self.batchnorm3(self.conv3(x))))

        # Flatten the output for fully connected layers
        x = x.view(-1, 64 * (6000 // 8))
        
        # Fully connected layers with dropout
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [4]:
def save_model(model, filepath):
    torch.save(model.state_dict(), filepath)
    print(f"Model saved successfully at {filepath}")

def load_model(model, filepath, device='cuda'):
    model.load_state_dict(torch.load(filepath))
    model.to(device)
    print(f"Model loaded successfully from {filepath}")
    return model

# UNBALANCED DATA

In [8]:
# Calculate class distribution in training data
class_distribution = Counter(y_tensor.tolist())

# Calculate percentage of each class
total_samples = len(y_tensor)
class_percentage = {label: count / total_samples * 100 for label, count in class_distribution.items()}

# Display class distribution
print("Class Distribution in Training Data:")
for label, count in class_distribution.items():
    print(f"Class {label}: {count} samples ({class_percentage[label]:.2f}%)")

Class Distribution in Training Data:
Class 0: 8073 samples (30.58%)
Class 1: 3268 samples (12.38%)
Class 2: 7000 samples (26.52%)
Class 3: 4704 samples (17.82%)
Class 5: 3355 samples (12.71%)


In [9]:
# Normal Training

learning_rate = 0.001
# Define the number of folds for stratified CV
num_folds = 5

# Initialize stratified KFold
stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# Training loop with stratified cross-validation
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(X_tensor, y_tensor)):
    print(f'Fold {fold + 1}/{num_folds}')
    
    # Initialize model, optimizer, criterion
    model = SleepStageClassifier()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    # Move model to appropriate device
    model.to(device)
    
    # Initialize data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.permute(0, 2, 1)
            #print(labels.dtype)
            # Convert target labels to torch.int64
            labels = labels.to(torch.int64)

            # Ensure model outputs have the correct data type
            outputs = model(inputs)
            outputs = outputs.float()  # Convert to float if needed
            optimizer.zero_grad()
            
            #print(outputs.dtype)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_correct_predictions = 0
    val_total_samples = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_inputs = val_inputs.permute(0, 2, 1)

            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct_predictions += (val_predicted == val_labels).sum().item()
            val_total_samples += val_labels.size(0)

    val_accuracy = val_correct_predictions / val_total_samples
    print(f'Validation Accuracy: {val_accuracy:.4f}')

print('Training finished')


Fold 1/5


  1%|          | 1/100 [02:31<4:10:16, 151.68s/it]

Epoch [1/100], Loss: 1.9361, Accuracy: 0.4228


  2%|▏         | 2/100 [05:21<4:25:31, 162.57s/it]

Epoch [2/100], Loss: 1.1596, Accuracy: 0.5352


...

Epoch [7/100], Loss: 0.8842, Accuracy: 0.6590


  8%|▊         | 8/100 [22:51<4:27:31, 174.47s/it]

Epoch [8/100], Loss: 0.8384, Accuracy: 0.6765


  8%|▊         | 8/100 [23:21<4:28:41, 175.23s/it]


KeyboardInterrupt: 

In [9]:
# Training with Stages

from torch.utils.data import DataLoader, Subset

# Function to save the state
def save_state(fold, model_state, optimizer_state):
    state = {
        'fold': fold,
        'model_state': model_state,
        'optimizer_state': optimizer_state
    }
    with open('training_state.pkl', 'wb') as f:
        pickle.dump(state, f)

# Function to load the state
def load_state():
    if os.path.exists('training_state.pkl'):
        with open('training_state.pkl', 'rb') as f:
            state = pickle.load(f)
            return state
    return None

learning_rate = 0.001
num_folds = 5

# Assuming X_tensor and y_tensor are your data tensors
# And dataset is your full dataset
# Example: dataset = MyCustomDataset(X_tensor, y_tensor)

# Initialize stratified KFold
stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# Load previous state if available
state = load_state()
if state:
    start_fold = state['fold']
else:
    start_fold = 0

# Training loop with stratified cross-validation
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(X_tensor, y_tensor)):
    if fold < start_fold:
        continue

    print(f'Fold {fold + 1}/{num_folds}')
    
    # Initialize model, optimizer, criterion
    model = SleepStageClassifier()
    model.to(device)  # Move model to device here

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    if state and fold == start_fold:
        model.load_state_dict(state['model_state'])
        optimizer.load_state_dict(state['optimizer_state'])
    
    
    # Initialize data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.permute(0, 2, 1)
            labels = labels.to(torch.int64)

            outputs = model(inputs)
            outputs = outputs.float()  # Convert to float if needed
            optimizer.zero_grad()
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_correct_predictions = 0
    val_total_samples = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_inputs = val_inputs.permute(0, 2, 1)

            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct_predictions += (val_predicted == val_labels).sum().item()
            val_total_samples += val_labels.size(0)

    val_accuracy = val_correct_predictions / val_total_samples
    print(f'Validation Accuracy: {val_accuracy:.4f}')

    # Save the state
    save_state(fold + 1, model.state_dict(), optimizer.state_dict())

    # Ask if the user wants to continue
    cont = input(f'Fold {fold + 1} completed. Do you want to continue to the next fold? (yes/no): ')
    if cont.lower() != 'yes':
        print('Training stopped.')
        break

print('Training finished')

Training finished


In [11]:
# Function to load the state
def load_state():
    if os.path.exists('training_state.pkl'):
        with open('training_state.pkl', 'rb') as f:
            state = pickle.load(f)
            return state
    return None

# Initialize the model and load the saved state
model = SleepStageClassifier()
state = load_state()
if state:
    model.load_state_dict(state['model_state'])
model.to(device)  # Move model to the appropriate device

SleepStageClassifier(
  (conv1): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=48000, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [12]:
save_model(model, "SleepStageClassifier.pth")

Model saved successfully at SleepStageClassifier.pth


# BALANCED DATA

In [7]:
# Define the batch size for processing
batch_size = 512

# Create an instance of B-SMOTE
bsmote = BorderlineSMOTE()

# Process each batch sequentially
balanced_batches_X = []
balanced_batches_y = []

for i in tqdm(range(0, len(X_tensor), batch_size)):
    batch_X = X_tensor[i:i+batch_size]
    batch_y = y_tensor[i:i+batch_size]

    # Reshape input data for B-SMOTE
    batch_X_reshaped = batch_X.reshape(-1, batch_X.shape[1] * batch_X.shape[2])

    # Apply B-SMOTE to balance the classes for the current batch
    batch_X_balanced, batch_y_balanced = bsmote.fit_resample(batch_X_reshaped, batch_y)

    # Reshape the balanced batch back to its original shape
    batch_X_balanced = batch_X_balanced.reshape(-1, batch_X.shape[1], batch_X.shape[2])

    # Store the balanced batch
    balanced_batches_X.append(batch_X_balanced)
    balanced_batches_y.append(batch_y_balanced)

# Concatenate the balanced batches
X_balanced = np.concatenate(balanced_batches_X, axis=0)
y_balanced = np.concatenate(balanced_batches_y, axis=0)

# Display the balanced data shapes
print("Balanced Dataset")
print("Input data shape:", X_balanced.shape)
print("Labels shape:", y_balanced.shape)


100%|██████████| 52/52 [01:59<00:00,  2.29s/it]


Balanced Dataset
Input data shape: (44877, 6000, 8)
Labels shape: (44877,)


In [8]:
# Calculate class distribution in training data
class_distribution = Counter(y_balanced.tolist())

# Calculate percentage of each class
total_samples = len(y_balanced)
class_percentage = {label: count / total_samples * 100 for label, count in class_distribution.items()}

# Display class distribution
print("Class Distribution in Training Data:")
for label, count in class_distribution.items():
    print(f"Class {label}: {count} samples ({class_percentage[label]:.2f}%)")

Class Distribution in Training Data:
Class 0: 10712 samples (23.87%)
Class 1: 10459 samples (23.31%)
Class 2: 9633 samples (21.47%)
Class 3: 6239 samples (13.90%)
Class 5: 7834 samples (17.46%)


In [9]:
from torch.utils.data import DataLoader, TensorDataset, Subset

# Convert balanced data to PyTorch tensors
X_balanced_tensor = torch.tensor(X_balanced, dtype=torch.float32)
y_balanced_tensor = torch.tensor(y_balanced, dtype=torch.long)  # Use torch.long for labels

learning_rate = 0.001

# Define dataset and dataloaders
batch_size = 32  # You can adjust the batch size as needed
num_epochs = 100
# Create TensorDataset
balanced_dataset = TensorDataset(X_balanced_tensor, y_balanced_tensor)

# Split into training and testing sets
train_size = int(0.8 * len(balanced_dataset))
test_size = len(balanced_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(balanced_dataset, [train_size, test_size])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the number of folds for stratified CV
num_folds = 5

# Function to save the state
def save_state(fold, model_state, optimizer_state):
    state = {
        'fold': fold,
        'model_state': model_state,
        'optimizer_state': optimizer_state
    }
    with open('training_state_balanced.pkl', 'wb') as f:
        pickle.dump(state, f)

# Function to load the state
def load_state():
    if os.path.exists('training_state_balanced.pkl'):
        with open('training_state_balanced.pkl', 'rb') as f:
            state = pickle.load(f)
            return state
    return None

# Load previous state if available
state = load_state()
if state:
    start_fold = state['fold']
else:
    start_fold = 0

# Initialize stratified KFold
stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

# Training loop with stratified cross-validation
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(X_balanced_tensor, y_balanced_tensor)):
    if fold < start_fold:
        continue

    print(f'Fold {fold + 1}/{num_folds}')
    
    # Initialize model, optimizer, criterion
    model = SleepStageClassifier()
    model.to(device)  # Move model to device here

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    if state and fold == start_fold:
        model.load_state_dict(state['model_state'])
        optimizer.load_state_dict(state['optimizer_state'])
    
    # Initialize data loaders with balanced dataset
    train_subset = Subset(balanced_dataset, train_indices)
    val_subset = Subset(balanced_dataset, val_indices)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size)

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.permute(0, 2, 1)
            labels = labels.to(torch.int64)

            outputs = model(inputs)
            outputs = outputs.float()  # Convert to float if needed
            optimizer.zero_grad()
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_correct_predictions = 0
    val_total_samples = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_inputs = val_inputs.permute(0, 2, 1)

            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct_predictions += (val_predicted == val_labels).sum().item()
            val_total_samples += val_labels.size(0)

    val_accuracy = val_correct_predictions / val_total_samples
    print(f'Validation Accuracy: {val_accuracy:.4f}')

    # Save the state
    save_state(fold + 1, model.state_dict(), optimizer.state_dict())

    # Ask if the user wants to continue
    cont = input(f'Fold {fold + 1} completed. Do you want to continue to the next fold? (yes/no): ')
    if cont.lower() != 'yes':
        print('Training stopped.')
        break

print('Training finished')

Fold 5/5


  1%|          | 1/100 [02:02<3:21:42, 122.24s/it]

Epoch [1/100], Loss: 0.2381, Accuracy: 0.9324


  2%|▏         | 2/100 [04:03<3:18:14, 121.37s/it]

Epoch [2/100], Loss: 0.1656, Accuracy: 0.9485


...

 99%|█████████▉| 99/100 [3:18:42<01:59, 119.95s/it]

Epoch [99/100], Loss: 0.0565, Accuracy: 0.9825


100%|██████████| 100/100 [3:20:42<00:00, 120.43s/it]

Epoch [100/100], Loss: 0.0540, Accuracy: 0.9830


Validation Accuracy: 0.9364
Training finished


In [10]:
save_model(model, "SleepStageClassifier_balanced.pth")

Model saved successfully at SleepStageClassifier_balanced.pth


# Testing Models

In [14]:
model = load_model(SleepStageClassifier(), "SleepStageClassifier_balanced.pth")
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
true_labels = []
predicted_labels = []



with torch.no_grad():  # Disable gradient computation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        inputs = inputs.permute(0, 2, 1)
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate predicted labels
        _, predicted = torch.max(outputs, 1)
        
        # Update true and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
f1New = f1_score(true_labels, predicted_labels, average=None)

#f1 = f1_score(true_labels, predicted_labels, average='none')

kappa = cohen_kappa_score(true_labels, predicted_labels)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'F1New-score: {f1New:.4f}')

print(f'Kappa coefficient: {kappa:.4f}')

Model loaded successfully from SleepStageClassifier_balanced.pth
Test Accuracy: 0.9790
Precision: 0.9752
Recall: 0.9784
F1-score: 0.9767


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [12]:
model = load_model(SleepStageClassifier(), "SleepStageClassifier.pth")
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
true_labels = []
predicted_labels = []



with torch.no_grad():  # Disable gradient computation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        inputs = inputs.permute(0, 2, 1)
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate predicted labels
        _, predicted = torch.max(outputs, 1)
        
        # Update true and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

#f1 = f1_score(true_labels, predicted_labels, average='none')

kappa = cohen_kappa_score(true_labels, predicted_labels)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'Kappa coefficient: {kappa:.4f}')

Model loaded successfully from SleepStageClassifier.pth
Test Accuracy: 0.8865
Precision: 0.8884
Recall: 0.8912
F1-score: 0.8885
Kappa coefficient: 0.8568
